In [ ]:
import os
import tarfile

import numpy as np
import plotly.graph_objects as go

import cuarray
import netchem
import netcalc

In [ ]:
#NETSCI_ROOT=<path to netsci root directory>
tutorial_directory = f'{NETSCI_ROOT}/tutorials'
os.chdir(tutorial_directory)

In [ ]:
tutorial_tarball = tarfile.open(f'{os.getcwd()}/tutorial.tar.gz')
tutorial_tarball.extractall()
tutorial_tarball.close()

In [ ]:
r = 0.9
gaussian_2D_mean = np.zeros(2)
gaussian_2D_cov = np.array([[1.0, r], [r, 1.0]])
gaussian_2D_size = 10000

In [ ]:
gaussian_2D = np.random.multivariate_normal(
    mean=gaussian_2D_mean,
    cov=gaussian_2D_cov,
    size=gaussian_2D_size,
).T.astype(np.float32)

In [ ]:
gaussian_2D_figure = go.Figure(
    data=go.Scatter(
    x=gaussian_2D[0],
    y=gaussian_2D[1],
    mode='markers',
    ))

In [ ]:
gaussian_2D_figure.show()

In [ ]:
X = cuarray.FloatCuArray()
X.fromNumpy2D(gaussian_2D)

In [ ]:
ab = cuarray.IntCuArray()

In [ ]:
ab.init(1, 2)
ab[0][0] = 0
ab[0][1] = 1

In [ ]:
I = cuarray.FloatCuArray()

In [ ]:
n = gaussian_2D_size
k = 4
xd = 2
d = 1
platform = netcalc.GPU_PLATFORM

In [ ]:
netcalc.mutualInformation(
    X=X,
    I=I, 
    ab=ab,
    k=k,
    n=n,
    xd=xd,
    d=d,
    platform=platform,
)

In [ ]:
print(I)

In [ ]:
analytic_I = -0.5*np.log(np.linalg.det(np.cov(gaussian_2D)))
print(analytic_I)

In [ ]:
pyro_network = netchem.Network()

In [ ]:
trajectory_file = f'{os.getcwd()}/pyro.dcd'
topology_file = f'{os.getcwd()}/pyro.pdb'
first_frame = 0
last_frame = 999
#stride=3
pyro_network.init(
    trajectoryFile=trajectory_file,
    topologyFile=topology_file,
    firstFrame=first_frame,
    lastFrame=last_frame,
    #stride=stride 
    """
    If the stride argument is provided, Network.init parses every 'stride' trajectory frame.
    """
)



In [ ]:
pyro_R = cuarray.FloatCuArray()
pyro_ab = cuarray.IntCuArray()

In [ ]:
pyro_num_nodes = pyro_network.numNodes()
pyro_num_node_pairs = pyro_num_nodes**2
pyro_ab.init(
    pyro_num_node_pairs,
    2,
)
for i in range(pyro_num_nodes):
    for j in range(pyro_num_nodes):
        pyro_node_pair_index = i*pyro_num_nodes + j
        pyro_ab[pyro_node_pair_index][0] = i
        pyro_ab[pyro_node_pair_index][1] = j

In [ ]:
pyro_n = pyro_network.numFrames()
pyro_d = 3
pyro_xd = 2
pyro_k = 4
pyro_platform = 0

In [ ]:
netcalc.generalizedCorrelation(
    X=pyro_network.nodeCoordinates(),
    R=pyro_R,
    ab=pyro_ab,
    k=pyro_k,
    n=pyro_n,
    d=pyro_d,
    xd=pyro_xd,
    platform=pyro_platform,
)

In [ ]:
pyro_R_np = pyro_R.toNumpy2D().reshape(
    pyro_num_nodes,
    pyro_num_nodes,
)

In [ ]:
np.save("pyro_R.npy", pyro_R_np.astype(np.float32))

In [ ]:
pyro_R_figure_x = [i for i in range(pyro_num_nodes)]
pyro_R_figure_y = [i for i in range(pyro_num_nodes)]

In [ ]:
pyro_R_figure = go.Figure(
    data=go.Heatmap(
        x=pyro_R_figure_x,
        y=pyro_R_figure_y,
        z=pyro_R_np,
        colorscale='jet',
        zsmooth='best',
    ),
)

In [ ]:
pyro_R_figure.show()